In [3]:
import ipynb.fs.full.data_operations as do
import ipynb.fs.full.backtester as bt
import pandas as pd
import time

In [4]:
ops = do.gspread_Operator()
#file = ops.get_client()
#ws = file.worksheet('Prev_Recos')
print('Reading all existing recos...')

prev_recos = ops.read_prev_recos()
prev_recos = prev_recos[['uuid', 'reco_date', 'analyst', 'house', 'stock_name', 'ticker', 'current_value', 'target_value', 'sl_value', 'timeframe', 'tested']]
testables = []
tested = 0

for idx, row in prev_recos.iterrows():
    if(tested % 10 == 0):
        time.sleep(3)                              #putthing this in place to allow google writes
    if(row['tested']==True):                       #Has this row been previously tested?
        continue
    try:
        testable = bt.ticker_Data.sanity_test(row)
        if (testable):
            scrip = bt.ticker_Data(ticker=f'{row['ticker']}.NS', start_date=row['reco_date'], in_price=row['current_value'], tgt_price=row['target_value'], sl_price=row['sl_value'])
            prices = scrip.get_price_data()
            prices.index = prices.index.tz_convert('Asia/Kolkata') 
            scrip.simple_win_or_loss(prices)
            if(scrip.outcome.get('result') != 'undet'):
                scrip.return_rate = (scrip.outcome_price/scrip.in_price - 1) * 100 
                scrip.return_horizon = (scrip.outcome.get('index').replace(tzinfo=None) - scrip.start_date_ts).days
                ops.write_backtesting_outcomes(idx+2, ['true', 'true', str(scrip.outcome), scrip.outcome.get('result'), scrip.outcome_price, scrip.return_rate, scrip.return_horizon, scrip.ticker])
            else:
                ops.write_backtesting_outcomes(idx+2, ['true', 'true', str(scrip.outcome), scrip.outcome.get('result'),'-', '-', '-', scrip.ticker])
            #print(f'Stock:{row['ticker']}, Record_Date:{row['reco_date']}, TGT: {row['target_value']}, SL: {row['sl_value']}, Results: {outcome}, Exit:{scrip.outcome_price}')
            tested += 1
            testables.append(testable)
            break
        else:
            ops.write_backtesting_outcomes(idx+2, ['TRUE', 'FALSE'])
            #ws.update_acell('K'+str(idx+1), 'TRUE')
            #ws.update_acell('L'+str(idx+1), 'FALSE')
            #ws.update(range_name='K1:L1', values=['True', 'False'])
            tested += 1
            testables.append(testable)
            #break
    except APIError as api:
        time.sleep(10)
        print('Waiting...')
    except Exception as e:
        print(f'For line {tested}, caught exception:{e}')
        import traceback
        traceback.print_exc()
    finally:
        continue

print(f'Executed tests for: {tested} rows from beginning.')

Reading all existing recos...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Executed tests for: 74 rows from beginning.
